# EXPLORING IMAGE CLASSIFICATION USING CNNs AND KERNEL METHODS

### Ho diviso il file nei rispettivi chunk di codice. Il dataset usato è CIFAR-10. Ho applicato data augmentation sul training set per avere diversità nel training e ridurre overfitting.

- **Parametri di regolarizzazione Res-Net**: dropout
- **Parametri di regolarizzazione SVM**: C
- **Parametri di regolarizzazione SVM con RFF**: C, $\gamma$


### Import packages

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
import torchvision.models as models

### Data Pre-Processing and Data Augmentation

In [ ]:
# Trasformazioni per il training (con data augmentation)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Flip orizzontale casuale
    transforms.RandomCrop(32, padding=4),  # Crop casuale con padding
    transforms.ToTensor(),  # Converti in tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizzazione
])

# Trasformazioni per validazione e test (senza data augmentation)
val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset per training e test
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True)

# Applica trasformazioni diverse per training e validazione
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
val_dataset = datasets.CIFAR10(root='./data', train=True, transform=val_test_transform, download=True)

# Applica trasformazione per il test
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=val_test_transform, download=True)

# Divisione del training set in training (80%) e validazione (20%)
train_size = int(0.8 * len(train_dataset))  # 80% per il training
val_size = len(full_train_dataset) - train_size  # 20% per la validazione
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader per batch processing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Training con shuffle
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # Validazione senza shuffle
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # Test senza shuffle

# Stampiamo il numero di esempi
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

### Set device to GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## ResNet-34 + Softmax

In [ ]:
#upload pre trained model
model = models.resnet34(weights='DEFAULT')
# Modifica dell'ultimo fully connected e applica  Dropout
model.fc = nn.Sequential(
    nn.Dropout(0.3), 
    nn.Linear(model.fc.in_features, 10)
)
model = model.to(device)

# Loss e ottimizzatore
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(model.parameters(), lr=0.001) 
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Funzioni di addestramento e validazione
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total


def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total


# Ciclo di addestramento
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)
    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Testare il modello
test_loss, test_acc = validate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")


## ResNet-34 + SVM end-to-end

In [ ]:
# Funzione di perdita personalizzata con Square Hinge Loss + L2 Regularization
def square_hinge_loss(outputs, targets, weights, C=10):
    
    # Creazione del tensore one-hot per i target (classi vere)
    targets_one_hot = torch.full_like(outputs, -1, device=outputs.device)  # Tutte le classi inizialmente a -1
    targets_one_hot[torch.arange(len(targets)), targets] = 1  # La classe corretta a +1
    
    # Calcolo del margine
    margins = 1 - targets_one_hot * outputs  # shape: (n_samples, n_classes)
    # Square hinge loss: max(0, margin)^2
    hinge_loss = torch.clamp(margins, min=0) ** 2  # shape: (n_samples, n_classes)
    
    # Media su tutti i campioni e classi
    hinge_loss = hinge_loss.mean()

    # Calcolo della regolarizzazione L2 (media dei quadrati dei pesi)
    reg_loss = torch.mean(torch.square(weights))
    # Loss complessiva con il parametro di penalizzazione C
    total_loss = C*hinge_loss+reg_loss
    

    return total_loss

# Caricamento del modello predefinito (ResNet34)
model = models.resnet34(weights='DEFAULT')

# Modifica dell'ultimo fully connected con Dropout
model.fc = nn.Sequential(
    nn.Dropout(0.3),  
    nn.Linear(model.fc.in_features, 10)
)
model = model.to(device)

# Ottimizzatore
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Funzioni di addestramento e validazione
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        
        # Output del modello (logit)
        outputs = model(inputs)

        # Pesi del layer finale
        readout_weights = model.fc[1].weight

        # Calcolare la square hinge loss con L2 regularization
        loss = square_hinge_loss(outputs, targets, readout_weights)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total

# Funzione di validazione
def validate(model, loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            # Pesi del layer finale
            readout_weights = model.fc[1].weight

            # Calcolare la square hinge loss con L2 regularization
            loss = square_hinge_loss(outputs, targets, readout_weights)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total



# Ciclo di addestramento
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, val_loader)
    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Testare il modello
test_loss, test_acc = validate(model, test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")


In [ ]:
# Testare il modello
test_loss, test_acc = validate(model, test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")

## ResNet-34 + Kernel SVM end-to-end using Random Fourier Features

In [ ]:
# Funzione per calcolare Random Fourier Features (RFF)
class RandomFourierFeatures(nn.Module):
    def __init__(self, input_dim, sigma=1):
        super(RandomFourierFeatures, self).__init__()
        self.input_dim = input_dim
        self.scale = sigma
        
        # Pesi fissi per RFF: un vettore di 512 pesi
        self.register_buffer("weights", torch.normal(mean=0, std=torch.sqrt(torch.tensor(sigma)), size=(input_dim,)))  # Vettore di 512 pesi
        self.register_buffer("bias", 2 * np.pi * torch.rand(input_dim))  # Bias casuali per l'RFF

    def forward(self, x):
        # Moltiplicazione element-wise dell'input x con il vettore di pesi
        projections = x * self.weights  # Moltiplicazione element-wise dell'input con i pesi
        
        # Aggiunta del bias
        projections = projections + self.bias  # Aggiunta del bias alla proiezione
        
        # Coseno della trasformazione
        projections = torch.cos(projections)  # Applichiamo il coseno
        
        # Moltiplichiamo per sqrt(2 / D), dove D è la dimensione dell'input (input_dim)
        return torch.sqrt(torch.tensor(2.0 / self.input_dim)) * projections

# Square hinge loss come definita in precedenza
def square_hinge_loss(outputs, targets, weights, C=1):
    targets_one_hot = torch.full_like(outputs, -1, device=outputs.device)
    targets_one_hot[torch.arange(len(targets)), targets] = 1

    margins = 1 - targets_one_hot * outputs
    hinge_loss = torch.clamp(margins, min=0) ** 2
    hinge_loss = hinge_loss.mean()

    reg_loss = torch.mean(torch.square(weights))
    total_loss = C * hinge_loss + reg_loss

    return total_loss

# Caricamento della ResNet34 predefinita
model = models.resnet34(weights='DEFAULT')

# Extract the embedding dimension from ResNet (which is 512 for ResNet34)
embedding_dim = model.fc.in_features 

# Define RFF layer, ensuring the output dimension stays the same as the input dimension
rff_layer = RandomFourierFeatures(embedding_dim)

# Replace the fully connected layer with a custom sequence (dropout, RFF, final linear layer)
model.fc = nn.Sequential(
    nn.Dropout(0.3),
    rff_layer,  # Apply Random Fourier Features
    nn.Linear(embedding_dim, 10)  # Final classification layer
)

# Move the model to the correct device
model = model.to(device)

# Ottimizzatore e scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Funzioni di addestramento e validazione
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        readout_weights = model.fc[-1].weight
        loss = square_hinge_loss(outputs, targets, readout_weights)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total

def validate(model, loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            readout_weights = model.fc[-1].weight
            loss = square_hinge_loss(outputs, targets, readout_weights)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total




# Ciclo di addestramento
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, val_loader)
    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Testare il modello
test_loss, test_acc = validate(model, test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")
